# MARIA MAGDALENE NAMAGANDA
# 2019/HD07/24853U
# Ace_class Kaggle assignment_1

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Importing some of the needed packages and libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns

from sklearn.model_selection import KFold
from pandas import read_csv
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier



> *Now i need to get rid of some unnecessary warnings by ignoring them*

In [ ]:
#To avoid unnecessary warnings, I will ignore them in the code below
import warnings
warnings.filterwarnings('ignore')

## Loading the datasets 
### There are two datasets given; 
1. AMP_TrainSet.csv and 
2. Test.csv  

In [ ]:
#Loading the datasets, 

Train = pd.read_csv("../input/amp-data-set/AMP_TrainSet.csv")
Test = pd.read_csv("../input/amp-data-set/Test.csv")


### For training the model, I will first use the training set which I will further split into the train and validation set. 
### Then I will test the model on the Test set provided to gauge its performance.

# Exploring my data
## Exploring data helps one understand what kind of data is given. That is to say knowing how much data it is, the shape, type, dimensions, missing values, data summary, correlation of attributes among others as am going to do in the following steps.

In [ ]:
#here, am trying to find the type of data
type(Train)
type(Test)
Train.dtypes, Test.dtypes

In [ ]:
# checking the dimensions of the data
# this retuns the number of rows and columns in the data

Train.shape, Test.shape

#this helps to know how big the data is in terms of rows and columns.
#also from here I can tell which data is labeled

## Data description
>For now, I will focus more on the train dataset because its what I will use to train the model 

In [ ]:
#getting a description of the train dataset
#description gives a summary of the data.

Train.describe()

In [ ]:
#looking at the first 5 entries of my data
Train.head()

## Class Distribution
> From the Train dataset, i see there is an extra 'CLASS' column which will be my validation set.
### I need to know how this class is distributed to guide me on what to do with it.

In [ ]:
#to see the class distribution, I will plot a bar graph
Train.groupby('CLASS').size().plot(kind='bar')
pyplot.show()

> I would like to know how many instaces i have for each class

In [ ]:
#getting the number of instances in each class
Train.groupby('CLASS').size()

### There are 1519 intances for each class which is proof for even distribution.
### From the above barplot and class instances, I can see that the classes are evenly distributed so no need to use smote.

# Data Visualisation
### Data visualization is the technique to present the data in a pictorial or graphical format. It enables one to analyze data visually. The data in a graphical format allows identification of new trends and patterns easily.
### Visualisation identifies the relationship between data points and variables.

## Density plots

> I chose to use density plots because they are better at determing the distribution shape as they are not affected by the number of bins as is the case with Histograms.

In [ ]:
#now plotting density subplots
#setting the figsize to 12 so that my graphs are not congested
Train.plot(kind='density', subplots = True, layout=(3,4), sharex= False, sharey= False, figsize=(12,12))
plt.show()


> From the above density subpots, I can see that most of the data follows a Gaussian distribution given some of the characteristics such as bell shaped curves and graphs being symmetrical about the mean.

## Box and whisker plots
> A box plot is a type of graph that displays a summary of a large amount of data in terms of the median, upper quartile, lower quartile, minimum and maximum data values.

> It handles large data easily, gives a clear summary and displays outliers.

In [ ]:
#plotting a box and whisker graph
#setting the figsize to 10 so that the plots are not congested.
Train.plot(kind='box', subplots = True, layout=(3,4), sharex= False, sharey= False, figsize=(10,10))
plt.show()

## Looking at the correlation of the data

> Correlation is the relationship between two variables. The commonly used method is Pearson's correlation coefficient. It assumes a normal distribution of the attributes involved.

> -1 shows full negative correlation, 

> +1 shows full negative correlation and 

> 0 shows no correlation at all.

In [ ]:
#first I will checkfor the pairwise correlation of the attributes.
Train.corr(method='pearson')

In [ ]:
#Then now am reviewing the inter-correlation of attributes using heatmap
#graphical representation
plt.figure(figsize=(6,6))
sns.heatmap(Train.corr(method='pearson'))

In [ ]:
#Ill also check the correlation in regards to the 'CLASS' attribute
Train.corr(method='pearson')['CLASS']

## Skewnwess of the data
> knowing data skewness allows one to perform data preparation and improve a model

> If Skewness value lies above +1 or below -1 then the data is highly skewed. 

> If skewness value lies between +0.5 and -0.5 then the data ids moderately skewed.

> If skewness is 0 then data is symmetrical

In [ ]:
#Checking out skewness of data
Train.skew().plot(kind='bar')

# Comparing Machine Learning Algorithms
>  This helps to choose the best model for the problem at hand.

>  Using resampling methods like cross validation, gives an estimate for how accurate each model may be on unseen data.

>  It is important to ensure that each algorithm is evaluated in the same way on the same data to avoid bias.

In [ ]:
#comparing different models to from which ill choose.


# load dataset

array = Train.values

#split the dataset 
X = array[:,0:11]  #X = Train.drop(columns=['CLASS'])
Y = array[:,11]   #Y = Train['CLASS']

# preparing models and adding them to a list
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBC', GradientBoostingClassifier()))
models.append(('EXT', ExtraTreesClassifier()))
models.append(('RTC', RandomForestClassifier()))
#models.append(('XGB', XGBClassifier)) 
#am commenting out XGB it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

# evaluating each model in turn
results = []
names = []

for name, model in models:
    kfold = KFold(n_splits=50, random_state=42)
    cv_results = cross_val_score(model, X, Y, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = (name, cv_results.mean(), cv_results.std())
    print(msg)

# boxplot algorithm comparison
fig = pyplot.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
pyplot.show()

> ## From the above algorithm comparison, I can see that Naive Bayes(NB), ExtraTreesClassifiers(EXT) and RandomForestClassifiers(RTC) are some of the best performing algorithms. 
> ## Am yet to find out the overall best valgorithm after prediction on the Test set

# Feature selection 
## Using recursive feature elimination
> Sometimes, you may asses a dataset and find out that you do not need to use all the given features. This can be because some of them are highly correlates or they are not in any way helpful in developing the model.
* It is therefore important to get rid of some features where applicable.

> For this data, I will use Recursive Feature Elimination(RFE)

In [ ]:
#feature selection using RFE
#first i will start with choosing 4 features
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

array = Train.values
X = array[:,0:11]
Y = array[:,11]
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 4)
fit = rfe.fit(X, Y)
print("Num Features: ", fit.n_features_)
print("Selected Features:", fit.support_)
print("Feature Ranking: ", fit.ranking_)

In [ ]:
#Calling out the column names so I can know which features am going to drop from RFE
Train.columns

## Using Feature importance

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

array = Train.values
X = array[:,0:11]
Y = array[:,11]
# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print(model.feature_importances_)

## If am to use 4 features and drop the rest
> I will assign the new dataset a variable 'New_Train4' after choosing the 4 features and dropping the rest.

In [ ]:
# I will call the new Train data with selected features New_Train4.
Train
New_Train4 = Train.drop(['FULL_Charge', 'FULL_AcidicMolPerc', 'FULL_DAYM780201', 'FULL_GEOR030101', 'AS_MeanAmphiMoment', 'AS_DAYM780201', 'AS_FUKS010112'], axis =1)

In [ ]:
#dropping the same features in the test dataset
New_Test4 = Test.drop(['FULL_Charge', 'FULL_AcidicMolPerc', 'FULL_DAYM780201', 'FULL_GEOR030101', 'AS_MeanAmphiMoment', 'AS_DAYM780201', 'AS_FUKS010112'], axis =1)

In [ ]:
#viewing the selected features
New_Train4.columns

In [ ]:
#now splitting data
#array = New_Train.values
X = New_Train4.values[:,0:4]
Y = New_Train4.values[:,4]
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# also train and test the model on Matthews correlation coefficient.
from sklearn.metrics import matthews_corrcoef

GS = GaussianNB()
GS.fit(X_Train,Y_Train)
pred = GS.predict(X_Test)

print("The result is: ",np.round(matthews_corrcoef(Y_Test,pred) *100,2)," Mathew's Coef")

## Using four features gives me a very low MCC and low overall score.
## I'll therefore consider using 8 features and see what score  I get.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

array = Train.values
X = array[:,0:11]
Y = array[:,11]
# feature extraction
model = LogisticRegression()
rfe = RFE(model, 8)
fit = rfe.fit(X, Y)
print("Num Features: ", fit.n_features_)
print("Selected Features:", fit.support_)
print("Feature Ranking: ", fit.ranking_)

In [ ]:
# I will call the new Train data with selected features New_Train8.
Train
New_Train8 = Train.drop(['FULL_AcidicMolPerc', 'FULL_DAYM780201', 'AS_DAYM780201'], axis =1)

In [ ]:
#dropping the same features in the test dataset
New_Test8 = Test.drop(['FULL_AcidicMolPerc', 'FULL_DAYM780201', 'AS_DAYM780201'], axis =1)

In [ ]:
#now splitting data
#array = New_Train.values


X = New_Train8.values[:,0:8]
Y = New_Train8.values[:,8]
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
#now creating a model and training it on 8 features
model = LogisticRegression(solver='liblinear', C=0.05, multi_class='ovr', random_state=30)
model.fit(X_Train, Y_Train)

In [ ]:
from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,
random_state=42)

In [ ]:
from sklearn.metrics import matthews_corrcoef

nv = GaussianNB()
nv.fit(X_Train,Y_Train)
pred = nv.predict(X_Test)

print("The result is: ",np.round(matthews_corrcoef(Y_Test,pred) *100,2)," Mathew's Coef")

## Standardising data
> This is useful to transform attributes with a Gaussian distribution and it workd better with rescaled data(also known as normalistaion where attributes are scaled into a range between 0 and 1)

# Below are some of the algorithms I will be using;

# LINEAR ALGORITHMS
1. Linear Regression
2. Logistic Regression
3. Linear Discriminant Analysis

# Logistic Regression using 8 features

In [ ]:
# here am using a dataset 'New_Train8' with 8 selected features
array = New_Train8.values
X = array[:,0:8]
Y = array[:,8]

kfold = KFold(n_splits=10, random_state=42) #spliiting my data into 10 folds and random_state of 42 for reproducibility
model = LogisticRegression() #calling out the prediction algorithm
results = cross_val_score(model, X, Y, cv=kfold) #estimating the model on new data and assigning it to results variable
print(results.mean())  #getting the mean of the accuracy scores from cross validation scores

model.fit(X,Y)
output = model.predict(New_Test8.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_logistic = pd.DataFrame(output)  #here we are converting the array into a pandas dataframe which will give a single column
maria_logistic.columns = ['CLASS'] #renaming the output column to 'CLASS'
maria_logistic.index.name = "Index"  #naming the index column as 'Index'
maria_logistic['CLASS']=maria_logistic['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_logistic.to_csv("maria_logistic.csv") #changing my output file as a 'csv' file

print(maria_logistic['CLASS'].unique()) #checking out the unique instances in the 'CLASS' column
print('False: ',maria_logistic.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_logistic.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# Applying Linear Dicriminant Analysis (LDA) using all features 

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]
num_folds = 10
kfold = KFold(n_splits=10, random_state=42) #spliiting my data into 10 folds and random_state of 42 for reproducibility
model = LinearDiscriminantAnalysis() #calling out the prediction algorithm
results = cross_val_score(model, X, Y, cv=kfold) #estimating the model on new data and assigning it to results variable
print(results.mean())  #getting the mean of the accuracy scores from cross validation scores

model.fit(X,Y)
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_LDA = pd.DataFrame(output)  #here we are converting the array into a pandas dataframe which will give a single column
maria_LDA.columns = ['CLASS'] #renaming the output column to 'CLASS'
maria_LDA.index.name = "Index"  #naming the index column as 'Index'
maria_LDA['CLASS']=maria_LDA['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_LDA.to_csv("maria_LDA.csv") #changing my output file as a 'csv' file

print(maria_LDA['CLASS'].unique())  #checking out the unique instances in the 'CLASS' column
print('False: ',maria_LDA.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_LDA.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# NON LINEAR ALGORITHMS
1. Naive Bayes
2. Support Vector Machines
3. K-Nearest Neighbours
4. Classification and Regression Trees
5. Learning Vector Quantization

# Using Naive Bayes and all the features

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]
test_size = 0.35 

kfold = KFold(n_splits=10) #spliiting my data into 10 folds 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42) #random_state of 42 for reproducibility

model = GaussianNB() #calling out the prediction algorithm
results = cross_val_score(model, X, Y, cv=kfold) #estimating the model on new data and assigning it to results variable
print(results.mean())  #getting the mean of the accuracy scores from cross validation scores


model.fit(X,Y)
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria2_bayes = pd.DataFrame(output)  #here we are converting the array into a pandas dataframe which will give a single column
maria2_bayes.columns = ['CLASS'] #renaming the output column to 'CLASS'
maria2_bayes.index.name = "Index"  #naming the index column as 'Index'
maria2_bayes['CLASS']=maria2_bayes['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria2_bayes.to_csv("maria2_bayes.csv") #changing my output file as a 'csv' file


print(maria2_bayes['CLASS'].unique()) #checking out the unique instances in the 'CLASS' column
print('False: ',maria2_bayes.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria2_bayes.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# Support Vector Machines algorithm using all features

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]
kfold = KFold(n_splits=10) #spliiting my data into 10 folds

model = SVC() #calling out the prediction algorithm
scoring = 'acuracy'
results = cross_val_score(model, X, Y, cv=kfold) #estimating the model on new data and assigning it to results variable
print(results.mean())  #getting the mean of the accuracy scores from cross validation scores

model.fit(X, Y)
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

mcc = matthews_corrcoef(model.predict(X), Y)
print('MCC: ',mcc)

maria_svc = pd.DataFrame(output)  #here we are converting the array into a pandas dataframe which will give a single column
maria_svc.columns = ['CLASS'] #renaming the output column to 'CLASS'
maria_svc.index.name = 'Index'  #naming the index column as 'Index'
maria_svc['CLASS'] = maria_svc['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'

maria_svc.to_csv('maria_svc.csv') #changing my output file as a 'csv' file

print(maria_svc['CLASS'].unique()) #checking out the unique instances in the 'CLASS' column
print('False: ',maria_svc.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_svc.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# Applying Classification and Regression Trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier
array = Train.values
X = array[:,0:11]
Y = array[:,11]
kfold = KFold(n_splits=10, random_state=42) #spliiting my data into 10 folds and random_state of 42 for reproducibility
model = DecisionTreeClassifier() #calling out the prediction algorithm
results = cross_val_score(model, X, Y, cv=kfold) #estimating the model on new data and assigning it to results variable
print(results.mean())  #getting the mean of the accuracy scores from cross validation scores


model.fit(X,Y)
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_tree = pd.DataFrame(output)  #here we are converting the array into a pandas dataframe which will give a single column
maria_tree.columns = ['CLASS'] #renaming the output column to 'CLASS'
maria_tree.index.name = "Index"  #naming the index column as 'Index'
maria_tree['CLASS']=maria_tree['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_tree.to_csv("maria_tree.csv") #changing my output file as a 'csv' file


print(maria_tree['CLASS'].unique()) #checking out the unique instances in the 'CLASS' column
print('False: ',maria_tree.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_tree.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# K-Nearest Neighbours

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]

kfold = KFold(n_splits=10, random_state=42) #spliiting my data into 10 folds and random_state of 42 for reproducibility
model = KNeighborsClassifier() #calling out the prediction algorithm
results = cross_val_score(model, X, Y, cv=kfold) #estimating the model on new data and assigning it to results variable
print(results.mean()) #getting the mean of the accuracy scores from cross validation scores

model.fit(X,Y) 
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_knn = pd.DataFrame(output) #here we are converting the array into a pandas dataframe which will give a single column
maria_knn.columns = ['CLASS'] #renaming the output column to 'CLASS' 
maria_knn.index.name = "Index" #naming the index column as 'Index'
maria_knn['CLASS']=maria_knn['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_knn.to_csv("maria_knn.csv") #changing my output file as a 'csv' file


print(maria_knn['CLASS'].unique()) #checking out the unique instances in the 'CLASS' column
print('False: ',maria_knn.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_knn.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# Esemble Algorithms
1. Boosting and Adaboost
2. Bagging and Random forest

# Applying Adaboost 

In [ ]:
array = Train.values

X = array[:,0:11]
Y = array[:,11]

kfold = KFold(n_splits=10, random_state=42)

model = AdaBoostClassifier(n_estimators=200, random_state=42) 
results = cross_val_score(model, X, Y, cv=kfold)

print(results.mean())


test_set = Test.values
model.fit(X, Y)
output = model.predict(test_set)

mcc = matthews_corrcoef(model.predict(X), Y)
print('MCC: ',mcc)

maria_AB= pd.DataFrame(output)
maria_AB.columns = ['CLASS']
maria_AB.index.name = 'Index'
maria_AB['CLASS'] = maria_AB['CLASS'].map({0.0:False, 1.0:True})

maria_AB.to_csv('maria_AB.csv')

print(maria_AB['CLASS'].unique())
print('False: ',maria_AB.groupby('CLASS').size()[0].sum())
print('True: ',maria_AB.groupby('CLASS').size()[1].sum())

# Applying Gradient Boosting Classifier

In [ ]:
array = Train.values

X = array[:,0:11]
Y = array[:,11]

#num_trees = 200
#seed =42

kfold = KFold(n_splits=10, random_state=42)
model = GradientBoostingClassifier(n_estimators=200, random_state=42)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

model.fit(X,Y) 
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_GBC = pd.DataFrame(output) #here we are converting the array into a pandas dataframe which will give a single column
maria_GBC.columns = ['CLASS'] #renaming the output column to 'CLASS' 
maria_GBC.index.name = "Index" #naming the index column as 'Index'
maria_GBC['CLASS']=maria_GBC['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_GBC.to_csv("maria_GBC.csv") #changing my output file as a 'csv' file


print(maria_GBC['CLASS'].unique()) #checking out the unique instances in the 
print('False: ',maria_GBC.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_GBC.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# Using Stochastic Gradient Boosting Classification

In [ ]:
array = Train.values
X = array[:,0:11]
Y = array[:,11]

kfold = KFold(n_splits=10, random_state=42)
model = XGBClassifier(n_estimators=200, random_state=42)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

model.fit(X,Y) 
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_XGB = pd.DataFrame(output) #here we are converting the array into a pandas dataframe which will give a single column
maria_XGB.columns = ['CLASS'] #renaming the output column to 'CLASS' 
maria_XGB.index.name = "Index" #naming the index column as 'Index'
maria_XGB['CLASS']=maria_XGB['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_XGB.to_csv("maria_XGB.csv") #changing my output file as a 'csv' file


print(maria_GBC['CLASS'].unique()) #checking out the unique instances in the 
print('False: ',maria_XGB.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_XGB.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column

# Using Extra Trees Classifier

In [ ]:
array = Train.values

X = array[:,0:11]
Y = array[:,11]

kfold = KFold(n_splits=10, random_state=42)
model = ExtraTreesClassifier(n_estimators=200) # (max_features=11)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())
                             
model.fit(X,Y) 
output = model.predict(Test.values) #testing the model on the test_set (Test.values)

from sklearn.metrics import matthews_corrcoef
mcc = matthews_corrcoef(model.predict(X),Y)
print('MCC:',mcc)
                       
maria_ETX = pd.DataFrame(output) #here we are converting the array into a pandas dataframe which will give a single column
maria_ETX.columns = ['CLASS'] #renaming the output column to 'CLASS' 
maria_ETX.index.name = "Index" #naming the index column as 'Index'
maria_ETX['CLASS']=maria_ETX['CLASS'].map({0.0:False, 1.0:True}) #converting '0.0' to' False' and '1.0' to 'True'
maria_ETX.to_csv("maria_ETX.csv") #changing my output file as a 'csv' file


print(maria_ETX['CLASS'].unique()) #checking out the unique instances in the 
print('False: ',maria_ETX.groupby('CLASS').size()[0].sum()) #summing up the '0' instances in the 'CLASS' column
print('True: ',maria_ETX.groupby('CLASS').size()[1].sum()) #summing up the '1' instances in the 'CLASS' column                           

# CONCLUSION
## From the algorithm comparisons and my prediction scores, Naive Bayes algorithm performed the best.
## I think this is because the data was following a Gaussian distribution(normally distributed) as seen earlier from the density subplots.